In [3]:
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

In [ ]:
from langchain_core.messages import BaseMessage
from langgraph.graph import MessagesState
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from typing_extensions import Optional, Annotated, List, Sequence, Dict
import operator

class AgentInputState(MessagesState):
    pass

class AgentState(MessagesState):
    """
    Main state for the full multi-agent research system.
    
    Extends MessagesState with additional fields for research coordination.
    Note: Some fields are duplicated across different state classes for proper
    state management between subgraphs and the main workflow.
    """

    # Input message generated from user conversation history
    input_message: List[Dict[str, str]]
    diagram_type: str
    mermaid_code: str

    # # Messages exchanged with the supervisor agent for coordination
    # supervisor_messages: Annotated[Sequence[BaseMessage], add_messages]
    # # Raw unprocessed research notes collected during the research phase
    # raw_notes: Annotated[list[str], operator.add] = []
    # # Processed and structured notes ready for report generation
    # notes: Annotated[list[str], operator.add] = []
    # # Final formatted research report
    # final_report: str

# ===== STRUCTURED OUTPUT SCHEMAS =====
class StartMessageState(BaseModel):
    """Schema for invoking the supervisor agent."""
    
    mermaid_code: str = Field(
        description="Mermaid code of the given image"
    )

In [ ]:
from datetime import datetime
from typing_extensions import Literal

from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessage, get_buffer_string
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
from prompts.mermaid_examples import first_prompt
import base64
from pathlib import Path

# Initialize model
# model = init_chat_model(model="openai:gpt-4.1", temperature=0.0)
model = init_chat_model(model="openai:gpt-4o-mini", temperature=0.0)


# ===== WORKFLOW NODES =====

def supervisor_node(state: AgentState) -> AgentState:
    """
    Supervisor agent node that reviews research progress and provides feedback.
    """
    # Combine all messages for context
    structured_output_model = model.with_structured_output(StartMessageState)
    
    # response = structured_output_model.invoke([
    #     HumanMessage(content=first_prompt.format(
    #         messages=get_buffer_string(messages=state["messages"])
    #     ))
    # ])
    response = structured_output_model.invoke(state["messages"])
    
    return {
        "mermaid_code": response.mermaid_code
    }



# Build the scoping workflow
teching_graph = StateGraph(AgentState, input_schema=AgentInputState)
teching_graph.add_node("supervisor_node", supervisor_node)
teching_graph.add_edge(START, "supervisor_node")
teching_graph.add_edge("supervisor_node", END)
teching_graph_workflow = teching_graph.compile()



graph TD;
    A[0] -->|10| B[Source Port];
    B -->|10| C[Destination Port];
    C -->|11| D[Sequence Number];
    D -->|7| E[Data];
    E -->|8| F[Reserved];
    F -->|6| G[Flag];
    G -->|7| H[Checksum];
    H -->|5| I[Flag];
    I -->|10| J[Data Offset];
    J -->|23| K[80];


In [ ]:
img_path = Path("40.jpg")
if not img_path.exists():
    raise FileNotFoundError("40.jpg not found in the current working directory")

b64 = base64.b64encode(img_path.read_bytes()).decode("ascii")
data_uri = f"data:image/jpeg;base64,{b64}"

first_message = HumanMessage(content=[
    {"type": "text", "text": "Give mermaid code of the given diagram."},
    {"type": "image_url", "image_url": {"url": data_uri}}
])

# result = teching_graph_workflow.invoke({"messages": [HumanMessage(content="I want to research the best coffee shops in")]}, config=thread)
result = teching_graph_workflow.invoke({"messages": first_message})
print(result["mermaid_code"])